# Computing metrics with event-level experiment data

This Lab contains a series of notebooks that transform event-level data collected during an Optimizely experiment into several useful experiment datasets:

- **Experiment units**: the individual units (usually website visitors or app users) that are exposed to a control or treatment in the course of an online experiment. 
- **Experiment events**: the conversion events, such as a button click or a purchase, that was influenced by an experiment. We compute this view by isolating the conversion events triggered during a finite window of time (called the attribution window) after a visitor has been exposed to an experiment treatment.
- **Metric observations**: a mapping of experiment units to metric-specific numerical outcomes observed during an experiment

In this notebook, we'll walk through an end-to-end workflow for computing a series of metrics with data collected by both Optimizely and a third party during an Optimizely Full Stack experiment.

## The experiment

We'll use simulated data from the following "experiment" in this notebook: 

Attic & Button, a popular imaginary retailer of camera equipment and general electronics, has seen increased shipping times for some of its orders due to logistical difficulties imposed by the COVID-19 pandemic. As a result, customer support call volumes have increased.  In order to inform potential customers and cut down on customer support costs, the company's leadership has decided to add an informative banner to the [atticandbutton.com](http://atticandbutton.com) homepage.

In order to measure the impact this banner has on customer support volumes and decide which banner message is most effective, the team at Attic & Button have decided to run an experiment with the following variations:

<table>
    <tr>
        <td>
            <img src="img/control.png" alt="Control" style="width:100%; padding-left:0px">
        </td>
        <td>
            <img src="img/message_1.png" alt="Message #1" style="width:100%; padding-right:0px">
        </td>
        <td>
            <img src="img/message_2.png" alt="Message #2" style="width:100%; padding-right:0px">
        </td>
    </tr>
    <tr>
        <td style="background-color:white; text-align:center">
            "control"
        </td>
        <td style="background-color:white; text-align:center">
            "message_1"
        </td>
        <td style="background-color:white; text-align:center">
            "message_2"
        </td>
    </tr>
</table>

## The challenge

Attic & Button's call centers are managed by a third party.  This third party shares call data with Attic & Button periodically in a [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) file, making it difficult to track customer support metrics on Optimizely's [Experiment Results Page](https://app.optimizely.com/l/QQbfVyRFQYGq-J57P-3XoQ?previousView=VARIATIONS&variation=email_button&utm_campaign=copy).

In this notebook, we'll use Optimizely Enriched Event Data and our third-party call data to compute a variety of metrics for our experiment, including "Support calls per visitor" and "Total call duration per visitor". 

## What we're going to do

1. Download Optimizely decision and conversion data for our experiment
2. Compute "experiment units" and "experiment events" datasets
3. Load customer support call log data compute an "experiment calls" dataset 
4. Compute a set of metrics with our experiment datasets
5. Compute sequential p-values and confidence intervals using Optimizely Stats Engine Service
6. Render a simple experiment results report

## Global parameters

The following global parameters are used to control the execution in this notebook.  These parameters may be overridden by setting environment variables prior to launching the notebook, e.g.:

```
export OPTIMIZELY_DATA_DIR=~/my_analysis_dir
```

In [ ]:
import os
from getpass import getpass
from IPython.display import clear_output

# This notebook requires an Optimizely API token.  
OPTIMIZELY_API_TOKEN = os.environ.get("OPTIMIZELY_API_TOKEN", "2:bqZXaNE24MFUlhyGFrKKY9DMA-G02xoou7fR0nQlQ3bT89uvjtF8")

# Uncomment the following block to enable manual API token entry
# if OPTIMIZELY_API_TOKEN is None:
#    OPTIMIZELY_API_TOKEN = getpass("Enter your Optimizely personal API access token:")

# Default path for reading and writing analysis data
OPTIMIZELY_DATA_DIR = os.environ.get("OPTIMIZELY_DATA_DIR", "./covid_test_data")

# Set environment variables
# These variables are used by other notebooks and shell scripts invoked
# in this notebook
%env OPTIMIZELY_DATA_DIR={OPTIMIZELY_DATA_DIR}
%env OPTIMIZELY_API_TOKEN={OPTIMIZELY_API_TOKEN}

clear_output()

## Download Optimizely Enriched Event data

This notebook relies (in part) on data downloaded from Optimizely's [Enriched Event Export Service](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-export).

The default input data for this notebook can be found in the in `covid_test_data` directory.  

If you have the [oevents](https://github.com/optimizely/oevents) command line tool installed and accessible in your`PATH` environment variable, you may uncomment the following commands to re-download this data. Note that this will require `OPTIMIZELY_API_TOKEN` to be set to the default value specified above.

We'll start by download [decision](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-data-specification#decisions-2) data collected during our experiment.  Each **decision** captures the moment a visitor was added to our experiment.

In [2]:
!oevents load --type decisions --experiment 18786493712 --date 2020-09-14

download: s3://optimizely-events-data/v1/account_id=596780373/type=decisions/date=2020-09-14/experiment=18786493712/part-00000-accac923-16d5-4105-b8e8-c0f702201dd1.c000.snappy.parquet to covid_test_data/type=decisions/date=2020-09-14/experiment=18786493712/part-00000-accac923-16d5-4105-b8e8-c0f702201dd1.c000.snappy.parquet


Next we'll download [conversion](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-data-specification#conversions-2) data collected during our experiment.  Each **conversion event** captures the moment a visitor took some action on our website, e.g. viewing our homepage, adding an item to their shopping cart, or making a purchase.

In [3]:
!oevents load --type events --date 2020-09-14

download: s3://optimizely-events-data/v1/account_id=596780373/type=events/date=2020-09-14/_SUCCESS to covid_test_data/type=events/date=2020-09-14/_SUCCESS
download: s3://optimizely-events-data/v1/account_id=596780373/type=events/date=2020-09-14/event=add_to_cart/part-00000-6bfb2530-7428-4aa2-b8b1-8cfc971adcbe.c000.snappy.parquet to covid_test_data/type=events/date=2020-09-14/event=add_to_cart/part-00000-6bfb2530-7428-4aa2-b8b1-8cfc971adcbe.c000.snappy.parquet
download: s3://optimizely-events-data/v1/account_id=596780373/type=events/date=2020-09-14/event=purchase/part-00000-6bfb2530-7428-4aa2-b8b1-8cfc971adcbe.c000.snappy.parquet to covid_test_data/type=events/date=2020-09-14/event=purchase/part-00000-6bfb2530-7428-4aa2-b8b1-8cfc971adcbe.c000.snappy.parquet
download: s3://optimizely-events-data/v1/account_id=596780373/type=events/date=2020-09-14/event=detail_page_view/part-00000-6bfb2530-7428-4aa2-b8b1-8cfc971adcbe.c000.snappy.parquet to covid_test_data/type=events/date=2020-09-14/event

## Load Decision and Conversion Data into Spark Dataframes

We'll use [PySpark](https://spark.apache.org/docs/latest/api/python/index.html) to transform data in this notebook. We'll start by creating a new local Spark session.

In [4]:
from pyspark.sql import SparkSession

num_cores = 1
driver_ip = "127.0.0.1"
driver_memory_gb = 1
executor_memory_gb = 2

# Create a local Spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config(f"local[{num_cores}]") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.repl.eagerEval.truncate", 120) \
    .config("spark.driver.bindAddress", driver_ip) \
    .config("spark.driver.host", driver_ip) \
    .config("spark.driver.memory", f"{driver_memory_gb}g") \
    .config("spark.executor.memory", f"{executor_memory_gb}g") \
    .getOrCreate()

Next we'll load our decision data into a Spark dataframe:

In [5]:
import os
from lib import util

decisions_dir = os.path.join(OPTIMIZELY_DATA_DIR, "type=decisions")

# load enriched decision data from disk into a new Spark dataframe
decisions = util.read_parquet_data_from_disk(
    spark_session=spark,
    data_path=decisions_dir,
    view_name="decisions"
)

Now we can write SQL-style queries against our `enriched_decisions` view.  Let's use a simple query to examine our data:

In [6]:
spark.sql("""
    SELECT
        *
    FROM
        decisions
    LIMIT 3
""")

uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,campaign_id,experiment_id,variation_id,attributes,user_ip,user_agent,referer,is_holdback,revision,client_engine,client_version,date,experiment
0244b48f-cd2c-45fe-86b5-accb0864aa9f,2020-09-14 11:38:10.022,2020-09-14 11:39:09.401,user_9763,-1361007105,596780373,18811053836,18786493712,18802093142,"[[$opt_bot_filtering, $opt_bot_filtering, custom, false], [$opt_enrich_decisions, $opt_enrich_decisions, custom, true...",162.227.140.251,python-requests/2.24.0,null,false,99,python-sdk,3.5.2,2020-09-14,18786493712
07c73a10-0575-4990-b8d3-c5750f5b6fa1,2020-09-14 11:35:00.323,2020-09-14 11:35:07.792,user_7889,-12601611,596780373,18811053836,18786493712,18818611832,"[[$opt_bot_filtering, $opt_bot_filtering, custom, false], [$opt_enrich_decisions, $opt_enrich_decisions, custom, true...",162.227.140.251,python-requests/2.24.0,null,false,99,python-sdk,3.5.2,2020-09-14,18786493712
07c9d321-c336-49f4-86d0-ff6fed1d5b49,2020-09-14 11:29:21.83,2020-09-14 11:29:28.71,user_4546,1353967797,596780373,18811053836,18786493712,18802093142,"[[$opt_bot_filtering, $opt_bot_filtering, custom, false], [$opt_enrich_decisions, $opt_enrich_decisions, custom, true...",162.227.140.251,python-requests/2.24.0,null,false,99,python-sdk,3.5.2,2020-09-14,18786493712


Next we'll load conversion data:

In [7]:
# oevents downloads conversion data into the type=events subdirectory
conversions_dir = os.path.join(OPTIMIZELY_DATA_DIR, "type=events")

# load conversion data from disk into a new Spark dataframe
converions = util.read_parquet_data_from_disk(
    spark_session=spark,
    data_path=conversions_dir,
    view_name="events"
)

Let's take a look at our data:

In [8]:
spark.sql("""
    SELECT
        *
    FROM
        events
    LIMIT 3
""")

uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,experiments,entity_id,attributes,user_ip,user_agent,referer,event_type,event_name,revenue,value,quantity,tags,revision,client_engine,client_version,date,event
01d16e55-c276-4147-b2ba-586ec55d18ee,2020-09-14 10:45:01.756,2020-09-14 10:45:20.535,user_7468,-294926545,596780373,"[[18803622799, 18805683213, 18774763028, false]]",18803874034,"[[$opt_bot_filtering, $opt_bot_filtering, custom, false], [$opt_enrich_decisions, $opt_enrich_decisions, custom, true...",162.227.140.251,python-requests/2.24.0,null,null,detail_page_view,0,null,0,"[product -> android, sku -> 456, category -> electronics]",91,python-sdk,3.5.2,2020-09-14,detail_page_view
0579989e-cf42-4f09-9994-35720ab4084e,2020-09-14 11:37:19.081,2020-09-14 11:38:18.692,user_9260,-1708947236,596780373,"[[18803622799, 18805683213, 18821642160, false], [18811053836, 18786493712, 18802093142, false]]",15776040040,"[[$opt_bot_filtering, $opt_bot_filtering, custom, false], [$opt_enrich_decisions, $opt_enrich_decisions, custom, true...",162.227.140.251,python-requests/2.24.0,null,null,add_to_cart,0,null,0,"[product -> android, price -> 799.99, sku -> 456, category -> electronics]",99,python-sdk,3.5.2,2020-09-14,add_to_cart
05c15bf4-f3a1-47e1-85ee-46c64b92caf8,2020-09-14 10:37:45.575,2020-09-14 10:38:12.498,user_3168,1630703036,596780373,"[[18803622799, 18805683213, 18774763028, false]]",18822540003,"[[$opt_bot_filtering, $opt_bot_filtering, custom, false], [$opt_enrich_decisions, $opt_enrich_decisions, custom, true...",162.227.140.251,python-requests/2.24.0,null,null,homepage_view,0,null,0,[],91,python-sdk,3.5.2,2020-09-14,homepage_view


## Compute some useful intermediate experiment datasets

In this section, we'll compute three useful intermediate experiment datasets:

1. Enriched decisions - Optimizely [decision](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-data-specification#decisions-2) data enriched with human-readable experiment and variation names.
2. Experiment Units - the individual units (usually website visitors or app users) that are exposed to a control or treatment in the course of a digital experiment.
3. Experiment Events - conversion events, such as a button click or a purchase, that were influenced by a digital experiment.

The following diagram illustrates how these datasets are used to compute _metric observations_ for our experiment:

![Transformations](img/transformations.png)

### Enriched decisions

First we'll use Optimizely's [Experiment API](https://library.optimizely.com/docs/api/app/v2/index.html#operation/get_experiment) to enrich our decision data with experiment and variation names.  This step makes it easier to build human-readable experiment reports with this data, as we will see below.

The code for enriching decision data can be found in the `enriching_decision_data.ipynb` notebook in this lab directory.

In [9]:
%run ./enriching_decision_data.ipynb

Successfully authenticated to Optimizely.
Found these experiment IDs in the loaded decision data:
    18786493712


### Experiment Units

**Experiment units** are the individual units that are exposed to a control or treatment in the course of an online experiment.  In most online experiments, subjects are website visitors or app users. However, depending on your experiment design, treatments may also be applied to individual user sessions, service requests, search queries, etc. 

<table>
    <tr>
        <td>
            <img src="img/transformations_1.png" alt="Experiment Units" style="width:100%; padding-left:0px">
        </td>
        <td>
            <img src="img/tables_1.png" alt="Experiment Units" style="width:100%; padding-left:0px">
        </td>
    </tr>
</table>

In [10]:
experiment_units = spark.sql(f"""
    SELECT
        *
    FROM (
        SELECT
            *,
            RANK() OVER (PARTITION BY experiment_id, visitor_id ORDER BY timestamp ASC) AS rnk
        FROM
            enriched_decisions
    )
    WHERE
        rnk = 1
    ORDER BY timestamp ASC
""").drop("rnk")
experiment_units.createOrReplaceTempView("experiment_units")

Let's examine our experiment unit dataset:

In [11]:
spark.sql("""
    SELECT
        visitor_id,
        experiment_name,
        variation_name,
        timestamp
    FROM
        experiment_units
    LIMIT 3
""")

visitor_id,experiment_name,variation_name,timestamp
user_0,covid_messaging_experiment,control,2020-09-14 11:21:40.177
user_1,covid_messaging_experiment,control,2020-09-14 11:21:40.279
user_2,covid_messaging_experiment,control,2020-09-14 11:21:40.381


Let's count the number of visitors in each experiment variation:

In [12]:
spark.sql("""
    SELECT 
        experiment_name,
        variation_name,
        count(*) as unit_count
    FROM 
        experiment_units
    GROUP BY 
        experiment_name,
        variation_name
    ORDER BY
        experiment_name ASC,
        variation_name ASC
""")

experiment_name,variation_name,unit_count
covid_messaging_experiment,control,3304
covid_messaging_experiment,message_1,3367
covid_messaging_experiment,message_2,3329


### Experiment Events

An **experiment event** is an event, such as a button click or a purchase, that was influenced by an experiment.  We compute this view by isolating the conversion events triggered during a finite window of time (called the _attribution window_) after a visitor has been exposed to an experiment treatment.

<table>
    <tr>
        <td>
            <img src="img/transformations_2.png" alt="Experiment Units" style="width:100%; padding-left:0px">
        </td>
        <td>
            <img src="img/tables_2.png" alt="Experiment Units" style="width:100%; padding-left:0px">
        </td>
    </tr>
</table>

In [13]:
# Create the experiment_events view
experiment_events = spark.sql(f"""
    SELECT
        u.experiment_id,
        u.experiment_name,
        u.variation_id,
        u.variation_name,
        e.*
    FROM
        experiment_units u INNER JOIN events e ON u.visitor_id = e.visitor_id
    WHERE
        e.timestamp BETWEEN u.timestamp AND (u.timestamp + INTERVAL 48 HOURS)
""")
experiment_events.createOrReplaceTempView("experiment_events")

Let's examine our Experiment Events dataset:

In [14]:
spark.sql("""
    SELECT
        timestamp,
        visitor_id,
        experiment_name,
        variation_name,
        event_name,
        tags,
        revenue
    FROM
        experiment_events
    LIMIT 10
""")

timestamp,visitor_id,experiment_name,variation_name,event_name,tags,revenue
2020-09-14 11:23:50.677,user_1283,covid_messaging_experiment,control,homepage_view,[],0
2020-09-14 11:23:50.883,user_1285,covid_messaging_experiment,message_1,homepage_view,[],0
2020-09-14 11:24:03.368,user_1408,covid_messaging_experiment,message_2,homepage_view,[],0
2020-09-14 11:24:21.053,user_1582,covid_messaging_experiment,message_1,homepage_view,[],0
2020-09-14 11:24:21.053,user_1582,covid_messaging_experiment,message_1,detail_page_view,"[product -> iphone, sku -> 123, category -> electronics]",0
2020-09-14 11:24:21.053,user_1582,covid_messaging_experiment,message_1,detail_page_view,"[product -> android, sku -> 456, category -> electronics]",0
2020-09-14 11:24:21.053,user_1582,covid_messaging_experiment,message_1,detail_page_view,"[product -> phone case, sku -> 789, category -> accessories]",0
2020-09-14 11:21:41.911,user_17,covid_messaging_experiment,message_1,homepage_view,[],0
2020-09-14 11:24:37.315,user_1742,covid_messaging_experiment,message_2,homepage_view,[],0
2020-09-14 11:24:56.09,user_1927,covid_messaging_experiment,message_1,detail_page_view,"[product -> android, sku -> 456, category -> electronics]",0


As above, let's count the number of events that were influenced by each variation:

In [15]:
spark.sql(f"""
    SELECT
        experiment_name,
        variation_name,
        event_name,
        count(*) as event_count
    FROM
        experiment_events
    GROUP BY
        experiment_name,
        variation_name,
        event_name
    ORDER BY
        experiment_name ASC,
        variation_name ASC,
        event_name ASC
""")

experiment_name,variation_name,event_name,event_count
covid_messaging_experiment,control,add_to_cart,326
covid_messaging_experiment,control,detail_page_view,1799
covid_messaging_experiment,control,homepage_view,3304
covid_messaging_experiment,control,purchase,326
covid_messaging_experiment,message_1,add_to_cart,338
covid_messaging_experiment,message_1,detail_page_view,2414
covid_messaging_experiment,message_1,homepage_view,3367
covid_messaging_experiment,message_1,purchase,338
covid_messaging_experiment,message_2,add_to_cart,446
covid_messaging_experiment,message_2,detail_page_view,2900


## Compute metric observations

**Metric observations** map each **experiment unit** to a specific numerical outcome observed during an experiment.  For example, in order to measure purchase conversion rate associated with each variation in an experiment, we can map each visitor to a 0 or 1, depending on whether or not they'd made at least one purchase during the attribution window in our experiment.

Unlike **experiment units** and **experiment events**, which can be computed using simple transformations,  **metric observations** are metric-dependent and can be arbitrarily complex, depending on the outcome you're trying to measure.

<table>
    <tr>
        <td>
            <img src="img/transformations_3.png" alt="Experiment Units" style="width:100%; padding-left:0px">
        </td>
        <td>
            <img src="img/tables_3.png" alt="Experiment Units" style="width:100%; padding-left:0px">
        </td>
    </tr>
</table>

In [16]:
from pyspark.sql.functions import lit, coalesce

def compute_metric_observations(
    metric_name, 
    raw_observations_df, 
    experiment_units_df,
    join_on="visitor_id",
    append_to=None,
    default_value=0
):
    """Compute a "metric observations" dataset for a given metric and (optionally) append it to an existing set of
    metric observations. Create (ore replace) a temporary view "observations" with the result.
    
    Parameters: 
        metric_name              - A string that uniquely identifies the metric for which observations are being computed,
                                   for example: "Purchase conversion rate"
                                   
        raw_observations_df      - A spark dataframe containing a set of raw observations for this metric. This dataframe 
                                   should contain two columns:
                                      visitor_id - a unique identifier for each unit
                                      observation - numerical outcome observered for this metric
                                   These metric observations will be joined with the provided experiment units dataframe
                                   so that the resulting dataset contains an observation for every unit. 
                                   
        experiment_units_df      - A spark dataframe containing the experiment units for which this metric should be
                                   computed.
                                   
        append_to (optional)     - A spark dataframe to which the resulting metric observation dataframe should be appended.
                                   If this is provided, the newly-combined dataframe will be returned.
                                    
        default_value (optional) - The default value to use for experiment units that do not appear in the raw observations
                                   dataframe.  If this is not provided, 0 is used.
    """

    merged_df = experiment_units_df \
                    .join(raw_observations_df, on=[join_on], how='left') \
                    .withColumn("_observation", coalesce('observation', lit(default_value))) \
                    .drop("observation") \
                    .withColumnRenamed("_observation", "observation") \
                    .withColumn("metric_name", lit(metric_name))

    if append_to is None:
        observations = merged_df
    else:
        observations = append_to.union(merged_df)
    
    observations.createOrReplaceTempView("observations")
    return observations

Now we'll define a set of observations by executing simple queries on our experiment events.  Each query computes a single _observation_ for each subject.

### Metric: Purchase conversion rate

In this query we measure for each visitor whether they made _at least one_ purchase. The resulting observation should be `1` if the visitor triggered the event in question during the _attribution window_ and `0` otherwise.  

Since _any_ visitor who triggered an appropriate experiment event should be counted, we can simply select a `1`. 

In [17]:
## Unique conversions on the "add to cart" event.
raw_purchase_conversion_rate_obs = spark.sql(f"""
    SELECT
        visitor_id,
        1 as observation
    FROM
        experiment_events
    WHERE
        event_name = 'purchase'
    GROUP BY
        visitor_id
""")
raw_purchase_conversion_rate_obs.toPandas().head(5)

,visitor_id,observation
0,user_5967,1
1,user_1434,1
2,user_3058,1
3,user_926,1
4,user_9069,1


We'll use our `add_observations` function to perform a left outer join between `experiment_units` and our newly-computed `add_to_cart` conversions.

In [18]:
observations = compute_metric_observations(
    "Purchase conversion rate",
    raw_purchase_conversion_rate_obs,
    experiment_units,
)

Let's take a look at our observations view:

In [19]:
observations.createOrReplaceTempView("observations")
spark.sql("""
    SELECT 
        metric_name,
        timestamp,
        visitor_id, 
        experiment_name, 
        variation_name, 
        observation 
    FROM 
        observations
    ORDER BY
        timestamp ASC
""")

metric_name,timestamp,visitor_id,experiment_name,variation_name,observation
Purchase conversion rate,2020-09-14 11:21:40.177,user_0,covid_messaging_experiment,control,0
Purchase conversion rate,2020-09-14 11:21:40.279,user_1,covid_messaging_experiment,control,0
Purchase conversion rate,2020-09-14 11:21:40.381,user_2,covid_messaging_experiment,control,0
Purchase conversion rate,2020-09-14 11:21:40.482,user_3,covid_messaging_experiment,message_1,0
Purchase conversion rate,2020-09-14 11:21:40.586,user_4,covid_messaging_experiment,message_1,0
Purchase conversion rate,2020-09-14 11:21:40.69,user_5,covid_messaging_experiment,message_2,0
Purchase conversion rate,2020-09-14 11:21:40.792,user_6,covid_messaging_experiment,message_2,0
Purchase conversion rate,2020-09-14 11:21:40.894,user_7,covid_messaging_experiment,message_2,0
Purchase conversion rate,2020-09-14 11:21:40.996,user_8,covid_messaging_experiment,message_1,0
Purchase conversion rate,2020-09-14 11:21:41.097,user_9,covid_messaging_experiment,control,0


Metric observations can be used to compute a variety of useful statistics.  Let's compute the value of our `purchase` conversion rate metric for all of the visitors in our experiment:

In [20]:
spark.sql("""
    SELECT
        metric_name,
        experiment_name,
        count(1) as unit_count,
        sum(observation),
        sum(observation) / (1.0 * count(1)) as metric_value
    FROM
        observations
    WHERE
        metric_name = "Purchase conversion rate"
    GROUP BY
        metric_name,
        experiment_name
""")

metric_name,experiment_name,unit_count,sum(observation),metric_value
Purchase conversion rate,covid_messaging_experiment,10000,555,0.05550000000000000


Now let's compute the `purchase` conversion rate broken down by experiment variation:

In [21]:
spark.sql("""
    SELECT
        metric_name,
        experiment_name,
        variation_name,
        count(1) as unit_count,
        sum(observation),
        sum(observation) / (1.0 * count(1)) as metric_value
    FROM
        observations
    WHERE
        metric_name = "Purchase conversion rate"
    GROUP BY
        metric_name,
        experiment_name,
        variation_name
""")

metric_name,experiment_name,variation_name,unit_count,sum(observation),metric_value
Purchase conversion rate,covid_messaging_experiment,message_1,3367,169,0.05019305019305019
Purchase conversion rate,covid_messaging_experiment,control,3304,163,0.04933414043583535
Purchase conversion rate,covid_messaging_experiment,message_2,3329,223,0.06698708320817062


### Metric: Product detail page views per visitor

In this query we the number of product detail page views per visitor

In [22]:
## Unique conversions on the "add_to_cart" event.
observations = compute_metric_observations(
    "Product detail page views per visitor",
    raw_observations_df = spark.sql("""
        SELECT
            visitor_id,
            count(1) as observation
        FROM
            experiment_events
        WHERE
            event_name = "detail_page_view"
        GROUP BY
            visitor_id
    """),
    experiment_units_df = experiment_units,
    append_to=observations
)

We can inspect our observations by counting the units and summing up the observations we've computed for each experiment in our dataset:

In [23]:
spark.sql("""
    SELECT 
        metric_name, 
        timestamp,
        experiment_name, 
        variation_id, 
        visitor_id, 
        observation 
    FROM 
        observations
    WHERE
        metric_name = "Product detail page views per visitor"
    LIMIT 10
""")

metric_name,timestamp,experiment_name,variation_id,visitor_id,observation
Product detail page views per visitor,2020-09-14 11:23:26.823,covid_messaging_experiment,18817551468,user_1048,0
Product detail page views per visitor,2020-09-14 11:23:59.303,covid_messaging_experiment,18817551468,user_1368,3
Product detail page views per visitor,2020-09-14 11:24:05.094,covid_messaging_experiment,18818611832,user_1425,3
Product detail page views per visitor,2020-09-14 11:24:41.774,covid_messaging_experiment,18817551468,user_1786,0
Product detail page views per visitor,2020-09-14 11:25:30.073,covid_messaging_experiment,18802093142,user_2262,0
Product detail page views per visitor,2020-09-14 11:25:38.915,covid_messaging_experiment,18802093142,user_2349,0
Product detail page views per visitor,2020-09-14 11:22:04.86,covid_messaging_experiment,18818611832,user_242,0
Product detail page views per visitor,2020-09-14 11:22:05.475,covid_messaging_experiment,18818611832,user_248,0
Product detail page views per visitor,2020-09-14 11:26:01.673,covid_messaging_experiment,18818611832,user_2573,0
Product detail page views per visitor,2020-09-14 11:26:38.674,covid_messaging_experiment,18818611832,user_2937,0


### Metric: Revenue from electronics purchases

In this query we compute the total revenue associated with electronics purchases made by our experiment subjects.

In [24]:
observations = compute_metric_observations(
    "Electronics revenue per visitor",
    raw_observations_df = spark.sql("""
        SELECT
            visitor_id,
            sum(revenue) as observation
        FROM 
            experiment_events
            LATERAL VIEW explode(tags) t
        WHERE
            t.key = "category" AND 
            t.value = "electronics" AND
            event_name = "purchase"
        GROUP BY
            visitor_id
    """),
    experiment_units_df = experiment_units,
    append_to=observations
)

Again, let's examine our observations:

In [25]:
spark.sql("""
    SELECT 
        metric_name, 
        timestamp,
        experiment_name, 
        variation_id, 
        visitor_id, 
        observation 
    FROM 
        observations
    WHERE
        metric_name = "Electronics revenue per visitor"
    LIMIT 20
""")

metric_name,timestamp,experiment_name,variation_id,visitor_id,observation
Electronics revenue per visitor,2020-09-14 11:23:26.823,covid_messaging_experiment,18817551468,user_1048,0
Electronics revenue per visitor,2020-09-14 11:23:59.303,covid_messaging_experiment,18817551468,user_1368,0
Electronics revenue per visitor,2020-09-14 11:24:05.094,covid_messaging_experiment,18818611832,user_1425,0
Electronics revenue per visitor,2020-09-14 11:24:41.774,covid_messaging_experiment,18817551468,user_1786,0
Electronics revenue per visitor,2020-09-14 11:25:30.073,covid_messaging_experiment,18802093142,user_2262,0
Electronics revenue per visitor,2020-09-14 11:25:38.915,covid_messaging_experiment,18802093142,user_2349,0
Electronics revenue per visitor,2020-09-14 11:22:04.86,covid_messaging_experiment,18818611832,user_242,0
Electronics revenue per visitor,2020-09-14 11:22:05.475,covid_messaging_experiment,18818611832,user_248,0
Electronics revenue per visitor,2020-09-14 11:26:01.673,covid_messaging_experiment,18818611832,user_2573,0
Electronics revenue per visitor,2020-09-14 11:26:38.674,covid_messaging_experiment,18818611832,user_2937,0


### Metric: Call center volume

We can use the same techniques to compute experiment metric using "external" data not collected by Optimizely.  We'll demonstrate by loading a CSV customer support call records.

We'll start by reading in our call center data:

In [26]:
# Read call center logs CSV into a pandas dataframe
df = pd.read_csv("call_data.csv")

# Display a sample of our call record data
df.head(5)

,user_id,call_start,call_duration_min
0,user_1007,9/15/2020 0:00:00,8.495311
1,user_1009,9/15/2020 3:00:00,2.162568
2,user_1014,9/15/2020 12:00:00,3.996617
3,user_1015,9/15/2020 16:00:00,6.886584
4,user_1017,9/15/2020 18:00:00,3.464795


Now let's make sure our call center data schema is compatible with the transformations we want to perform.

In [27]:
# Convert "call start" timestamp strings to datetime objects
df["timestamp"] = pd.to_datetime(df.call_start)

# Rename the "user_id" column to "visitor_id" to match our decision schema
df = df.rename(columns={"user_id" : "visitor_id"})

# Convert pandas to spark dataframe
call_records = spark.createDataFrame(df)

# Create a temporary view so that we can query using SQL
call_records.createOrReplaceTempView("call_records")

# Display a sample of our call record data
spark.sql("SELECT * FROM call_records LIMIT 5")

visitor_id,call_start,call_duration_min,timestamp
user_1007,9/15/2020 0:00:00,8.495310611,2020-09-15 00:00:00
user_1009,9/15/2020 3:00:00,2.16256821,2020-09-15 03:00:00
user_1014,9/15/2020 12:00:00,3.99661667,2020-09-15 12:00:00
user_1015,9/15/2020 16:00:00,6.886583842,2020-09-15 16:00:00
user_1017,9/15/2020 18:00:00,3.4647948310000003,2020-09-15 18:00:00


Now let's transform our call center logs into "experiment calls" using the attribution logic we used above to compute "experiment events":

In [28]:
# Create the experiment_calls view
experiment_calls = spark.sql(f"""
    SELECT
        u.experiment_id,
        u.experiment_name,
        u.variation_id,
        u.variation_name,
        e.*
    FROM
        experiment_units u INNER JOIN call_records e ON u.visitor_id = e.visitor_id
    WHERE
        e.timestamp BETWEEN u.timestamp AND (u.timestamp + INTERVAL 48 HOURS)
""")
experiment_calls.createOrReplaceTempView("experiment_calls")

Now we can compute metric observations for call center calls and duration!

In [29]:
# Count the number of support phone calls per visitor
observations = compute_metric_observations(
    "Customer support calls per visitor",
    raw_observations_df = spark.sql("""
        SELECT
            visitor_id,
            count(1) as observation
        FROM 
            experiment_calls
        GROUP BY
            visitor_id
    """),
    experiment_units_df = experiment_units,
    append_to=observations
)

# Count the number of support phone calls per visitor
observations = compute_metric_observations(
    "Total customer support minutes per visitor",
    raw_observations_df = spark.sql("""
        SELECT
            visitor_id,
            sum(call_duration_min) as observation
        FROM 
            experiment_calls
        GROUP BY
            visitor_id
    """),
    experiment_units_df = experiment_units,
    append_to=observations
)

## Computing metric values for experiment cohorts

We can slice and dice our metric observation data to compute metric values for different experiment cohorts.  Here are some examples:

### Computing metric values per variation

Let's start by computing metric values broken down by experiment variation.

In [30]:
# Compute metric values broken down by experiment variation
spark.sql("""
    SELECT
        metric_name,
        experiment_name,
        variation_name,
        count(1) as unit_count,
        sum(observation),
        sum(observation) / (1.0 * count(1)) as metric_value
    FROM
        observations
    GROUP BY
        metric_name,
        experiment_name,
        variation_name
    ORDER BY
        metric_name,
        experiment_name,
        variation_name
""")

metric_name,experiment_name,variation_name,unit_count,sum(observation),metric_value
Customer support calls per visitor,covid_messaging_experiment,control,3304,1115.0,0.33746973365617433
Customer support calls per visitor,covid_messaging_experiment,message_1,3367,1109.0,0.32937332937332936
Customer support calls per visitor,covid_messaging_experiment,message_2,3329,1115.0,0.3349354160408531
Electronics revenue per visitor,covid_messaging_experiment,control,3304,1.4499837E7,4388.570520581114
Electronics revenue per visitor,covid_messaging_experiment,message_1,3367,1.4959831E7,4443.07425007425
Electronics revenue per visitor,covid_messaging_experiment,message_2,3329,1.9899777E7,5977.704115349955
Product detail page views per visitor,covid_messaging_experiment,control,3304,1799.0,0.5444915254237288
Product detail page views per visitor,covid_messaging_experiment,message_1,3367,2414.0,0.7169587169587169
Product detail page views per visitor,covid_messaging_experiment,message_2,3329,2900.0,0.871132472213878
Purchase conversion rate,covid_messaging_experiment,control,3304,163.0,0.04933414043583535


### Computing metric values for a visitor segment

We can filter metric observations by visitor attributes in order to compute metric values for a particular segment.

In [31]:
# Compute metric values broken down by customer segment
spark.sql("""
    SELECT
        metric_name,
        experiment_name,
        variation_name,
        attrs.value as browser,
        count(1) as unit_count,
        sum(observation),
        sum(observation) / (1.0 * count(1)) as metric_value
    FROM
        observations
        LATERAL VIEW explode(attributes) AS attrs
    WHERE
        attrs.name = "browser"
    GROUP BY
        metric_name,
        experiment_name,
        variation_name,
        attrs.value
    ORDER BY
        metric_name,
        experiment_name,
        variation_name,
        attrs.value
""")

metric_name,experiment_name,variation_name,browser,unit_count,sum(observation),metric_value
Customer support calls per visitor,covid_messaging_experiment,control,chrome,1651,566.0,0.34282253179890976
Customer support calls per visitor,covid_messaging_experiment,control,firefox,1094,353.0,0.3226691042047532
Customer support calls per visitor,covid_messaging_experiment,control,safari,559,196.0,0.35062611806797855
Customer support calls per visitor,covid_messaging_experiment,message_1,chrome,1723,590.0,0.3424260011607661
Customer support calls per visitor,covid_messaging_experiment,message_1,firefox,1085,342.0,0.3152073732718894
Customer support calls per visitor,covid_messaging_experiment,message_1,safari,559,177.0,0.31663685152057247
Customer support calls per visitor,covid_messaging_experiment,message_2,chrome,1695,569.0,0.335693215339233
Customer support calls per visitor,covid_messaging_experiment,message_2,firefox,1121,377.0,0.33630686886708294
Customer support calls per visitor,covid_messaging_experiment,message_2,safari,513,169.0,0.32943469785575047
Electronics revenue per visitor,covid_messaging_experiment,control,chrome,1651,7719913.0,4675.90127195639


## Computing sequential statistics with Optimizely's Stats Services

We're working on launching a set of Stats Services that can be used to perform sequential hypothesis testing on metric observation data.  You can learn more about these services and request early access [here](optimizely.com/solutions/data-teams).

## Writing our datasets to disk

We'll write our experiment units, experiment events, and metric observations datasets to disk so that they may be used for other analysis tasks.

In [32]:
from lib import util

experiment_units_dir = os.path.join(OPTIMIZELY_DATA_DIR, "type=experiment_units")
util.write_parquet_data_to_disk(experiment_units, experiment_units_dir, partition_by="experiment_id")

experiment_events_dir = os.path.join(OPTIMIZELY_DATA_DIR, "type=experiment_events")
util.write_parquet_data_to_disk(experiment_events, experiment_events_dir, partition_by=["experiment_id", "event_name"])

metric_observations_dir = os.path.join(OPTIMIZELY_DATA_DIR, "type=metric_observations")
util.write_parquet_data_to_disk(observations, metric_observations_dir, partition_by=["experiment_id", "metric_name"])

## How to run this notebook

This notebook lives in the [Optimizely Labs](http://github.com/optimizely/labs) repository.  You can download it and everything you need to run it by doing one of the following
- Downloading a zipped copy of this Lab directory on the [Optimizely Labs page](https://www.optimizely.com/labs/computing-experiment-subjects/)
- Downloading a [zipped copy of the Optimizely Labs repository](https://github.com/optimizely/labs/archive/master.zip) from Github
- Cloning the [Github respository](http://github.com/optimizely/labs)

Once you've downloaded this Lab directory (on its own, or as part of the [Optimizely Labs](http://github.com/optimizely/labs) repository), follow the instructions in the `README.md` file for this Lab.